# mig_meow demonstration notebook

This Notebook is defined as a brief demonstration of mig_meow, which is a python package for defining MEOW workflows in conjunction with the MiG. This notebook will not fully explain the concepts of mig_meow, Pattterns, Recipes, MiG, or VGrids. MEOW and its related concepts are the subject of a currently being written paper which will be linked here once done.  

Its intended use is that this notebook would be launched through the MiG's own Jupyter spawner, which ensures maximum functionality. If you are not running this notebook through the MiG Jupyter spawner you may still define MEOW workflows and run a local model, but the Jupyter widget functionality will be reduced.

In [1]:
import mig_meow as meow
meow.info()

ver: 0.15
mig_meow has been imported correctly. 
Managing Event Oriented Workflows is a paradigm used for defining event based workflows. It is designed primarily to work with IDMC, a MiG implementation available at the University of Copenhagen. 


## Widgets

The main portal for defining and manipulating Patterns and Recipes is the workflow widget. Here, new Patterns and Recipes can be created, editted or deleted. Data can be saved to a local disk, or written to the appropriate VGrid on the MiG. If data is exported to the MiG using the 'Export' button then it will be used to potentiall schedule new Jobs and start a Workflow. 

Even though 'some_vgrid_name' does not exist we can still use the widget to inspect locally defined MEOW objects and to see how they interact. These objects can be loaded into the widget by hitting the 'Load' button. Note that for the widget to be trully useful in displaying a complete Workflow you should specify the expected outputs of a Pattern job. This is not necessary, and in fact is ignored by the MEOW definition, but will noticbly improve the readability of the preview. It follows therefore that this image is not necessarily an accurate representation of the actual produced workflow.

In [ ]:
meow.workflow_widget(vgrid='some_vgrid_name')

These is also a widget for monitoring the jobs scheduled on the MiG from Patterns. Note that this function will not work if you are testing outside of the MiG as you will lack essential setup variables for establishing a connection to the MiG.

In [ ]:
try:
    meow.monitor_widget(vgrid='some_vgrid_name')
except OSError:
    print('Cannot start the monitor widget outside of the MiG.')

## Local operations

Even if we're not connected to the MiG we can still use the workflow_widget in code cell 2 to create workflows. We can also do this outside of the widget. Here we can load in some pre-definded Patterns and Recipes.

In [ ]:
r = meow.read_dir()

patterns = r['patterns']
recipes = r['recipes']

print(patterns.keys())
print(recipes.keys())

We could also read in individual Patterns or Recipes if we didn't need all of the predefined ones.

In [ ]:
r = meow.read_dir_pattern('third_choo')
print(r.name)

r = meow.read_dir_recipe('choo')
print(r['name'])

We can also define new Patterns programmatically using the inbuilt object functions, and can save them locally.

In [ ]:
p = meow.Pattern('test_pattern')
p.add_single_input('in_file', 'text/*')
# Note the recipe does not need to exist yet.
p.add_recipe('appending')
# This ensures we've made a valid Pattern, and will provide feedback if not.
p.integrity_check()

# If we want to re-use this Pattern we can save it to disk.
meow.write_dir_pattern(p)

Recipes are ultimatly Jupyter notebooks, and so are mainly defined in their own notebooks. Here we can register the notebook 'recipe_examples.ipynb' as a recipe. We can also save this as a Recipe.

In [ ]:
ex = meow.register_recipe('recipe_example.ipynb', name='appending')

# We should check that the notebook we're read in is valid
meow.is_valid_recipe_dict(ex)

r = meow.write_dir_recipe(ex)

If we are operating with a MiG VGrid we can export our Patterns and Recipes to the VGrid.

In [ ]:
try:
    res = meow.write_vgrid(patterns, recipes, 'some_vgrid_name')
    print(res)
except OSError:
    print('Cannot write to VGrid if you are not running from outside the MiG')

## Running a local MEOW Workflow

Although mig_meow is designed primarly to interact with the MiG, it can run its own local workflow runner. This should not be taken as a complete workflow system as it is very bare-bones, and has none of the advanced features expected from a modern scientific workflow engine. However, it can take Pattern and Recipe definitions and apply an event driven workflow model to a given data directory (and its sub-directories). This can give some indication of how a MEOW workflow might work in prcatice, or could be used to test initial workflow explorations. 

As a final note, this workflow runner is intended only as a model for illustrating MEOW workflows. It is not considered safe for deployment in any real system. 

In [2]:
import os

# First get some Patterns and Recipes to work with. We're going to use the Patterns and Recipes provided with mig_meow 
# though any valid Patterns and Recipes can be used.
response = meow.read_dir()

patterns = response['patterns']
recipes = response['recipes']

print('Got patterns: %s' % list(patterns.keys()))
print('Got recipes: %s' % list(recipes.keys()))

# We're going to run the local runner in it's own fresh directory to illustrate exactly what new files the workflow makes, 
# though this isn't stricly necessary.
local_vgrid = 'local_workflow_base'

if os.path.exists(local_vgrid):
    os.rmdir(local_vgrid)
os.mkdir(local_vgrid)

# We can now start our workflow runner. Note that the '3' is the number of parallel workers in the workflow runner.
meow.start_local_workflow(local_vgrid, patterns, recipes, 3)

Got patterns: ['second_mult', 'third_choo', 'adder', 'first_mult']
Got recipes: ['add', 'mult', 'choo']


AttributeError: module 'mig_meow' has no attribute 'start_local_workflow'